In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [1]:
import pinecone
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

# get api key from app.pinecone.io for .env file
lines = open("../.env").read().splitlines()
for line in lines:
    if line.startswith("PINECONE_API_KEY"):
        api_key = line.split("=")[1]
        break
api_key = api_key or os.environ.get("PINECONE_API_KEY")
# find your environment next to the api key in pinecone console
env = os.environ.get("PINECONE_ENVIRONMENT") or "gcp-starter"

pinecone.init(api_key=api_key, environment=env)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import csv
import uuid
from tqdm.notebook import tqdm
import numpy as np

# delete index
index_name = "test-1"
PINECONE_MAX_K = 10_000


def try_random_search_for_dim(
    dim, tot_count=100_000, batch_size=200, marker_key="exported"
):
    pinecone.delete_index(index_name)
    pinecone.list_indexes()
    # only create index if it doesn't exist
    index, successful_count = upsert_index(index_name, dim, tot_count, batch_size)
    # upsert vectors
    # index.upsert(vectors)
    # how many vectors are in the index?
    vec_count = index.describe_index_stats()["total_vector_count"]
    print(f"Vector count(according to Pinecone): {vec_count}")
    all_ids = set()
    # search for vectors similar to a random vector
    # tqdm with total as vec_count to show progress bar, and len(all_ids) to show how many unique ids have been found
    pbar = tqdm(total=successful_count)
    # clear the progress bar
    for i in range(100):
        query_vector = np.random.rand(dim).tolist()
        results = index.query(
            vector=query_vector,
            include_values=False,
            top_k=PINECONE_MAX_K,
            filter={marker_key: {"$ne": True}},
        )
        ids = set(result["id"] for result in results["matches"])
        new_ids = ids - all_ids
        new_ids_list = list(new_ids)
        print(f"Found {len(new_ids)} new ids")
        # fetch and upsert with marker_key = True in metadata
        for j in tqdm(range(0, len(new_ids_list), batch_size)):
            batch_ids = list(new_ids_list)[j : j + batch_size]
            data = index.fetch(ids=batch_ids)
            batch_vectors = data["vectors"]
            upsert_data = []
            for id, vector_data in batch_vectors.items():
                if "metadata" not in vector_data:
                    vector_data["metadata"] = {}
                vector_data["metadata"][marker_key] = True
                cur_vec = pinecone.Vector(
                    id=id,
                    values=vector_data["values"],
                    metadata=vector_data["metadata"],
                )
                if vector_data.get("sparseValues"):
                    cur_vec.sparse_values = vector_data["sparseValues"]
                upsert_data.append(cur_vec)
            index.upsert(upsert_data)

        prev = len(all_ids)
        all_ids.update(ids)
        curr = len(all_ids)
        pbar.update(curr - prev)
        # pbar.set_description(f"Iteration {i+1}")
        if len(all_ids) == successful_count:
            break
    print(
        f"dim={dim}, {len(all_ids)} unique ids found, out of {vec_count} total vectors in {i+1} tries"
    )
    # append the above to a csv file

    with open("./results2.csv", "a") as f:
        writer = csv.writer(f)
        writer.writerow([dim, len(all_ids), vec_count, i + 1, successful_count])


def upsert_index(index_name, dim, tot_count, batch_size, gen_uuid=False):
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(name=index_name, dimension=dim, metric="cosine")
        print(f"Created index {index_name}")

    # now connect to the index
    index = pinecone.Index(index_name)
    # create list of random 1 d vectors to upsert, random metadata from keys a,b,c,d set to X or Y randomly
    # random numbers are between 0 and 1

    vectors = [
        {
            "id": str(i) if not gen_uuid else str(uuid.uuid4()),
            "values": np.random.rand(dim).tolist(),
            "metadata": {
                "a": "X" if np.random.rand(1) > 0.5 else "Y",
                "b": "X" if np.random.rand(1) > 0.5 else "Y",
                "c": "X" if np.random.rand(1) > 0.5 else "Y",
                "d": "X" if np.random.rand(1) > 0.5 else "Y",
            },
        }
        for i in range(tot_count)
    ]
    print(f"Created {len(vectors)} vectors")
    successful_count = 0
    for i in tqdm(range(0, len(vectors), batch_size)):
        batch_vectors = vectors[i : i + batch_size]
        try:
            index.upsert(batch_vectors)
            successful_count += len(batch_vectors)
        except Exception as e:
            print(f"Error upserting vectors at index {i}")
            print(e)
            break
    print(f"Successfully upserted {successful_count} vectors")
    return index, successful_count


# print(len(all_ids))

In [4]:
upsert_index("str-id-index", 100, 1000, 200, gen_uuid=True)

Created index str-id-index
Created 1000 vectors


  0%|          | 0/5 [00:00<?, ?it/s]

Successfully upserted 1000 vectors


(<pinecone.index.Index at 0x105ceca30>, 1000)

In [56]:
try_random_search_for_dim(384, tot_count=100_000, batch_size=400, marker_key="exported")

Created index test-1
Created 100000 vectors


  0%|          | 0/250 [00:00<?, ?it/s]

Successfully upserted 100000 vectors
Vector count(according to Pinecone): 97200


  0%|          | 0/100000 [00:00<?, ?it/s]

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '134', 'x-pinecone-request-latency-ms': '412', 'date': 'Wed, 27 Dec 2023 11:51:12 GMT', 'x-envoy-upstream-service-time': '76', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":9,"message":"Starter index record limit reached. Current number: 100000, records in request: 400, limit: 100000","details":[]}


In [5]:
# create a csv file with the following columns
# dim, unique_ids_found, total_vectors, iterations
# iterate through a range of dimensions, and try random search for each
# append the results to the csv file
# import csv
# with open(f'./results2.csv', 'a') as f:
#     writer = csv.writer(f)
#     writer.writerow(['dim', 'unique_ids_found', 'total_vectors', 'iterations', 'successful_count'])

for tot_count in range(1_000, 100_000, 10_000):
    for dim in [768, 1536]:
        print(f"Trying dim={dim}, tot_count={tot_count}")
        try_random_search_for_dim(dim, tot_count=tot_count)

Trying dim=768, tot_count=1000
Created index test-1
Created 1000 vectors


  0%|          | 0/5 [00:00<?, ?it/s]

Successfully upserted 1000 vectors
Vector count(according to Pinecone): 0


  0%|          | 0/1000 [00:00<?, ?it/s]

dim=768, 1000 unique ids found, out of 0 total vectors in 11 tries
Trying dim=1536, tot_count=1000
Created index test-1
Created 1000 vectors


...

Error upserting vectors at index 50600
(500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '150', 'x-pinecone-request-latency-ms': '493', 'date': 'Fri, 22 Dec 2023 08:31:33 GMT', 'x-envoy-upstream-service-time': '53', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":13,"message":"We were unable to process your request. If the problem persists, please contact us at https://support.pinecone.io","details":[]}

Successfully upserted 50600 vectors
Vector count(according to Pinecone): 50000


  0%|          | 0/50600 [00:00<?, ?it/s]

dim=1536, 50159 unique ids found, out of 50000 total vectors in 100 tries
Trying dim=768, tot_count=81000
Created index test-1
Created 81000 vectors


  0%|          | 0/405 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
# write a file with numbers from 0 to 3199, one per row
with open("ids.txt", "w") as f:
    for i in range(3200):
        f.write(f"{i}\n")

In [14]:
%pip install aiobotocore==2.12.3 aioitertools==0.11.0 botocore==1.34.69 fsspec==2023.12.2 gcsfs==2023.12.2.post1 google-api-core==2.19.0 google-cloud-core==2.4.1 google-cloud-storage==2.16.0 google-crc32c==1.5.0 google-resumable-media==2.7.0 googleapis-common-protos==1.63.0 pinecone-client==3.2.2 pinecone-datasets==0.7.0 proto-plus==1.23.0 pyarrow==11.0.0 pydantic==1.10.15 s3fs==2023.12.2 wrapt==1.16.0

  Obtaining dependency information for aiobotocore==2.12.3 from https://files.pythonhosted.org/packages/71/86/bbe79b24d4603c65a67e405661092c2fe0fa9b14e78dc8270bc83777412e/aiobotocore-2.12.3-py3-none-any.whl.metadata
  Obtaining dependency information for botocore==1.34.69 from https://files.pythonhosted.org/packages/c6/78/919e50b633035216dfb68627b1a4eac1235148b89b34a28f07fd99e8ac17/botocore-1.34.69-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core==2.19.0 from https://files.pythonhosted.org/packages/2d/ed/e514e0c59cdf1a469b1a1ab21b77698d0692adaa7cbc920c3a0b287e8493/google_api_core-2.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-storage==2.16.0 from https://files.pythonhosted.org/packages/cb/e5/7d045d188f4ef85d94b9e3ae1bf876170c6b9f4c9a950124978efc36f680/google_cloud_storage-2.16.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for googleapis-common-protos==1.63.0 from https://files.pythonhosted.org/pa

In [2]:
import pinecone_datasets

ds = pinecone_datasets.load_dataset("ANN_Fashion-MNIST_d784_euclidean")

In [3]:
df_l = pinecone_datasets.list_datasets(as_df=True)
df_l

,name,created_at,documents,queries,source,license,bucket,task,dense_model,sparse_model,description,tags,args
0,ANN_DEEP1B_d96_angular,2023-03-10 14:17:01.481785,9990000,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_DEEP1B_d96_angular,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
1,ANN_Fashion-MNIST_d784_euclidean,2023-03-10 14:17:01.481785,60000,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_Fashion-MNIST_d...,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
2,ANN_GIST_d960_euclidean,2023-03-10 14:17:01.481785,1000000,1000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_GIST_d960_eucli...,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
3,ANN_GloVe_d100_angular,2023-03-10 14:17:01.481785,1183514,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_GloVe_d100_angular,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
4,ANN_GloVe_d200_angular,2023-03-10 14:17:01.481785,1183514,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_GloVe_d200_angular,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
5,ANN_GloVe_d25_angular,2023-03-10 14:17:01.481785,1183514,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_GloVe_d25_angular,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
6,ANN_GloVe_d50_angular,2023-03-10 14:17:01.481785,1183514,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_GloVe_d50_angular,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
7,ANN_LastFM_d64_angular,2023-03-10 14:17:01.481785,292385,50000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_LastFM_d64_angular,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
8,ANN_MNIST_d784_euclidean,2023-03-10 14:17:01.481785,60000,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_MNIST_d784_eucl...,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None
9,ANN_NYTimes_d256_angular,2023-03-10 14:17:01.481785,290000,10000,https://github.com/erikbern/ann-benchmarks,None,gs://pinecone-datasets-dev/ANN_NYTimes_d256_an...,ANN,"{'name': 'ANN benchmark dense model', 'tokeniz...","{'name': None, 'tokenizer': None}",None,None,None


In [17]:
df_l[df_l["name"] == "amazon_toys_quora_all-MiniLM-L6-bm25"].iloc[0].dense_model

{'name': 'sentence-transformers/all-MiniLM-L6-v2',
 'tokenizer': None,
 'dimension': 384}

In [5]:
amz_ds = pinecone_datasets.load_dataset("amazon_toys_quora_all-MiniLM-L6-bm25")

In [10]:
amz_ds.head()

,id,values,sparse_values,metadata,blob
0,eac7efa5dbd3d667f26eb3d3ab504464,"[0.0077547780238091946, -0.02774387039244175, ...","{'indices': [2182291806, 4287202515, 148124445...",{'amazon_category_and_sub_category': 'Hobbies ...,{'text': 'Hornby 2014 Catalogue (Hornby): Pr...
1,b17540ef7e86e461d37f3ae58b7b72ac,"[0.002257382730022073, -0.03035414218902588, 0...","{'indices': [2118423442, 2177509083, 224097760...",{'amazon_category_and_sub_category': 'Hobbies ...,{'text': 'FunkyBuys® Large Christmas Holiday E...
2,348f344247b0c1a935b1223072ef9d8a,"[-0.003095218911767006, 0.016020774841308594, ...","{'indices': [2349888478, 3814962844, 310417642...",{'amazon_category_and_sub_category': 'Hobbies ...,{'text': 'CLASSIC TOY TRAIN SET TRACK CARRIAGE...
3,e12b92dbb8eaee78b22965d2a9bbbd9f,"[-0.024034591391682625, -0.048526741564273834,...","{'indices': [2182291806, 719182917, 1942275469...",{'amazon_category_and_sub_category': 'Hobbies ...,{'text': 'HORNBY Coach R4410A BR Hawksworth Co...
4,e33a9adeed5f36840ccc227db4682a36,"[-0.07078640908002853, 0.009733847342431545, 0...","{'indices': [2182291806, 2415375917, 369727517...",{'amazon_category_and_sub_category': 'Hobbies ...,{'text': 'Hornby 00 Gauge 0-4-0 Gildenlow Salt...


In [14]:
len(amz_ds.documents.iloc[0]["values"])

384

In [6]:
import pinecone

pc = pinecone.Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [21]:
from pinecone import ServerlessSpec

pc.create_index(
    name="amazon-toys-quora-all-minilm-l6-bm25",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-west-2"),
)

In [29]:
index = pc.Index("amazon-toys-quora-all-minilm-l6-bm25")

for batch in amz_ds.iter_documents(batch_size=100):
    # remove null values from the batch
    # break
    # Metadata value must be a string, number, boolean or list of string
    for doc in batch:
        for key, value in doc["metadata"].items():
            if not isinstance(value, (str, int, float, bool, list)):
                doc["metadata"][key] = str(value)
    try:
        index.upsert(vectors=batch)
    except Exception as e:
        print(e)
        break

(400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 15 May 2024 18:42:51 GMT', 'Content-Type': 'application/json', 'Content-Length': '92', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '178', 'x-pinecone-request-id': '1010414569894152647', 'x-envoy-upstream-service-time': '7', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Sparse vector size 2211 exceeds the maximum size of 1000","details":[]}



In [32]:
amz_ds.to_pinecone_index(
    index_name="amazon-toys-quora-all-minilm-l6-bm25",
    should_create_index=True,
    serverless=True,
    cloud="aws",
    region="us-west-2",
)

sending upsert requests:   0%|          | 0/10000 [00:00<?, ?it/s]

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 15 May 2024 19:16:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '92', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '341', 'x-pinecone-request-id': '4313797377394377488', 'x-envoy-upstream-service-time': '6', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Sparse vector size 2211 exceeds the maximum size of 1000","details":[]}


In [48]:
pinecone.delete_index(name="fashion")
pinecone.create_index(name="test-1", dimension=384)

In [13]:
dataset.to_pinecone_index(
    "fashion",
    api_key=api_key,
    environment=env,
    should_create_index=False,
    batch_size=300,
)

sending upsert requests:   0%|          | 0/60000 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pinecone/core/utils/__init__.py:113: FutureWarning: DEPRECATED since v2.2.1 [Will be removed in v3.0.0]: Deprecated type passed in 'values'. The ability to pass a numpy ndarray as part of a dictionary argument to upsert() will be removed in a future version of the pinecone client. To remove this warning, use the numpy.ndarray.tolist method to convert your ndarray into a python list before calling upsert().
  warnings.warn(message, FutureWarning)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pinecone/core/utils/__init__.py:113: FutureWarning: DEPRECATED since v2.2.1 [Will be removed in v3.0.0]: Deprecated type passed in 'values'. The ability to pass a numpy ndarray as part of a dictionary argument to upsert() will be removed in a future version of the pinecone client. To remove this warning, use the numpy.ndarray.tolist method to convert your ndarray into a python list before calling upsert().
  warnings.warn(message, FutureWarni

{'upserted_count': 60000}

In [25]:
dataset.head()

,id,values,sparse_values,metadata,blob
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",None,None,None
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",None,None,None
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",None,None,None
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 33.0,...",None,None,None
4,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",None,None,None


In [22]:
index = pinecone.Index("fashion")

In [4]:
# index.upsert_from_dataframe(dataset)

In [23]:
# 60k dim vector
input_vector = np.zeros(784).tolist()
input_vector

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [44]:
res = index.query(
    vector=input_vector,
    filter={"exported_vectoio_a": {"$ne": True}},
    top_k=10000,
    # include_metadata=True,
    # include_values=True,
)
len(res["matches"])

10000

In [45]:
# index.fetch(ids=[x['id'] for x in res['matches'][:2]])


In [46]:
ids_all = [x["id"] for x in res["matches"]]
for i in range(0, len(ids_all), 300):
    ids = ids_all[i : i + 100]
    fetched = index.fetch(ids=ids)["vectors"]
    for i in fetched:
        if "metadata" in fetched[i]:
            print(fetched[i]["metadata"])

{'exported_vectorio_8a64e': True}


In [47]:
tasks = []
for i in res["matches"]:
    tasks.append(
        index.update(
            id=i["id"],
            set_metadata={"exported_vectoio_a": True},
            async_req=True,
        )
    )
[p.get() for p in tasks]

KeyboardInterrupt: 

In [ ]:
ids = [x["id"] for x in res["matches"]]

In [50]:
fv = res["matches"][0]["metadata"]

In [14]:
fv

NameError: name 'fv' is not defined

In [57]:
# delete the vectors from 99000 to 99999
index = pinecone.Index("test-1")
ids = [str(i) for i in range(99000, 100000)]
index.delete(ids=ids)

{}

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="/Users/dhruvanand/Code/vector-io/.env")

True

In [5]:
%pip install pinecone-client -U

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/9c/64/081b55a33e492fc181524a955c2b65ba8a628dbc1bb897e65b723c7b7ffc/pinecone_client-4.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.0.2
    Uninstalling pinecone-client-3.0.2:
      Successfully uninstalled pinecone-client-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-datasets 0.6.2 requires pinecone-client<3.0.0,>=2.2.2, but you have pinecone-client 4.0.0 which is incompatible.
pinecone-datasets 0.6.2 requires pyarrow<12.0.0,>=11.0.0, but you have pyarrow 14.0.2 which is incompatible.
pinecone-datasets 0.6.2 requires pydantic<2.0.0,>=1.10.5, but you have pydantic 2.7.1 which is incompa

In [1]:
from pinecone import Pinecone
import os


pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [2]:
pc.list_indexes()

{'indexes': [{'dimension': 100,
              'host': 'str-id-index-tkao8rd.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'str-id-index',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1,
                               'source_collection': ''}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 784,
              'host': 'fashion-mnist-tkao8rd.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'euclidean',
              'name': 'fashion-mnist',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 768,
              'host': 'wikipedia-1k-cohere-openai-embeddings-tkao8rd.svc.apw5-4e34-81fa.pinecone.io'

In [3]:
index = pc.Index("test-list-points")

In [4]:
%pip show pinecone-client

Name: pinecone-client
Version: 4.0.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/dhruvanand/miniforge3/lib/python3.10/site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: pinecone-datasets, vdf_io
Note: you may need to restart the kernel to use updated packages.


In [13]:
len(pc.list_indexes().index_list["indexes"])

10

In [22]:
from halo import Halo

with Halo(text="pc.list_indexes()", spinner="dots"):
    list_index = pc.list_indexes().index_list["indexes"]
for index_dict in list_index:
    index = index_dict["name"]
    index_obj = pc.Index(index)
    print(index, index_obj.describe_index_stats())
    try:
        with Halo(spinner="dots"):
            ids_list = [idx for idx in index_obj.list()]
        print(f"{len(ids_list)=}")
    except Exception:
        print(f"not supported for {index}")

str-id-indexdexes()⠙ pc.list_indexes()⠹ pc.list_indexes()⠸ pc.list_indexes()⠼ pc.list_indexes()⠴ pc.list_indexes()⠦ pc.list_indexes()⠧ pc.list_indexes()⠇ pc.list_indexes() {'dimension': 100,
 'index_fullness': 0.01,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}
not supported for str-id-index
fashion-mnist {'dimension': 784,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 29001}},
 'total_vector_count': 29001}
len(ids_list)=291 ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ 

KeyboardInterrupt: 

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c7525f0> (for post_run_cell), with arguments args (<ExecutionResult object at 10c7ff880, execution_count=22 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10c7fff70, raw_cell="from halo import Halo
with Halo(text="pc.list_inde.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X36sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c66ed40> (for post_run_cell), with arguments args (<ExecutionResult object at 10c7ff880, execution_count=22 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10c7fff70, raw_cell="from halo import Halo
with Halo(text="pc.list_inde.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X36sZmlsZQ%3D%3D> result=None>,),kwargs {}:


...

TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c9b1bd0> (for post_run_cell), with arguments args (<ExecutionResult object at 10c7ff880, execution_count=22 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10c7fff70, raw_cell="from halo import Halo
with Halo(text="pc.list_inde.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X36sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c9b0ca0> (for post_run_cell), with arguments args (<ExecutionResult object at 10c7ff880, execution_count=22 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10c7fff70, raw_cell="from halo import Halo
with Halo(text="pc.list_inde.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X36sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [28]:
dbpedia_id = pc.Index("dbpedia-entities")

for idx in tqdm(
    dbpedia_id.list(limit=100),
    total=dbpedia_id.describe_index_stats()["total_vector_count"],
):
    tqdm.write(f"{len(idx)=} {idx=}")

  0%|          | 0/15874 [00:00<?, ?it/s]

len(idx)=100 idx=['0005f81d-5b8f-2dbb-c4a1-5fa1da67cb4e', '0009aa7e-a4bf-01c2-16cf-0c636d2fc50a', '000e4c95-9375-fd9e-3a48-94cf63fee83e', '00119e17-e793-245a-9505-466e752b9ceb', '001a8694-b48b-45df-4d74-ad0fc3da716b', '001efd72-b045-39f9-de04-5a9b6f47103d', '0021da2a-c3f4-cba0-d49e-f6bcc6744c51', '0021f040-a399-56d4-bb2b-dba71688a756', '002da849-0098-0d4b-b9f9-0614b78cfbfa', '002e2375-dd20-dec1-871f-68407a5be21c', '0036e797-7a11-58c9-5d65-c6309e21c4f0', '0037533d-8691-d108-b6f1-f0b3a3462ca4', '0037943b-def0-461c-c2f0-01782e977364', '0039f60c-47c0-9ec1-9837-ba6d7f1fedd1', '0040131c-10be-8877-8788-795ac2db6632', '0044647e-8ed3-4317-cb76-259de9d031df', '00470095-5cdf-6892-9501-7b969e564ff9', '004ae37f-f63e-308a-5d00-b4e8567c8411', '004cb027-3fa5-b3ea-52db-0529b20ead02', '005742f7-e6c6-6f28-781b-e904fcea756a', '0057ba8a-44f4-5a30-1caa-3dd48cab933d', '005b4624-c6b3-7913-eac7-2536fa4d8e81', '005ee0a9-d0da-1588-3427-21a4fb3350ed', '00686d2b-3f10-efe2-227a-d4f2fd7f3279', '006bf4b6-91b1-1295-0e

KeyboardInterrupt: 

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c7525f0> (for post_run_cell), with arguments args (<ExecutionResult object at 10ced7d30, execution_count=28 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10ced7f40, raw_cell="from tqdm.notebook import tqdm

dbpedia_id = pc.In.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X40sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

...

TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c9b1bd0> (for post_run_cell), with arguments args (<ExecutionResult object at 10ced7d30, execution_count=28 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10ced7f40, raw_cell="from tqdm.notebook import tqdm

dbpedia_id = pc.In.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X40sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x10c9b0ca0> (for post_run_cell), with arguments args (<ExecutionResult object at 10ced7d30, execution_count=28 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 10ced7f40, raw_cell="from tqdm.notebook import tqdm

dbpedia_id = pc.In.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/upsert_pinecone.ipynb#X40sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given